In [2]:
import pandas as pd
import geopandas as geopd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)
import ipynb
%run helper.ipynb
aggrcols = ['municipality_code','state_code','year']

In [3]:
def smr(smr_rate_cols=['age_group', 'location_type']):
    dfb = pd.read_csv('../dataset/Mexico/bc_data/BDmortalidadCAMAbase.csv')
    dfb = process_df(dfb,2)
    dfb['year'] = dfb['year_of_occurrence']
    dfb['age_group'] = dfb['age_group'].apply(update_age_group)
    dfb = dfb[ (dfb['population']>0) & (dfb['year']>2008) ]
    dfb['ndeaths'] = 1
    dfb2 = dfb.groupby(['state_code','municipality_code','age_group','year','location_type']).agg({
        'ndeaths': 'sum',
    }).reset_index()
    df = pd.merge(dfb, dfb2, on=['state_code','municipality_code','age_group','year','location_type'])
    df['count'] = 1
    df = standard_mortality(df, smr_rate_cols, [])
    return df

In [4]:
def emissions():
    dfem = pd.read_csv("../dataset/Mexico/emissions/emissions.csv")
    dfem = process_df(dfem)
    ### concentration of carcinogenic emissions ###
    dfem['concentration'] = dfem.apply(lambda row: calculate_concentration_sum2(row, ['1','2A']), axis=1)
    df_emissions = dfem.groupby(['state_code','municipality_code','year']).agg({
        'concentration': 'sum',
    }).reset_index()
    return df_emissions

In [3]:
def water_sources():
    dfw = pd.read_csv("../dataset/Mexico/water_sources/water_sources.csv")
    dfw = process_df(dfw)
    gdf = geopd.read_file("../dataset/Mexico/geometry/2023_1_00_MUN/2023_1_00_MUN.shp")
    gdf_water = geopd.GeoDataFrame(
        dfw, geometry=geopd.points_from_xy(dfw.x_coordinate,dfw.y_coordinate, crs="epsg:4326",
    ))
    combgdf = geopd.sjoin(gdf_water, gdf, how="left", predicate="intersects")
    dfws = combgdf
    dfws['CVE_MUN'] = dfws['CVE_MUN'].astype(str).str.lstrip('0').replace('', '0').astype(int)
    dfws['municipality_code'] = dfws['CVE_MUN']
    ### concentration of carcinogenic substances ###
    concentration_columns = [col for col in dfws.columns if 'concentration' in col]
    chemical_columns = [col for col in dfws.columns if '_concentration' not in col]
    dfws[chemical_columns] = dfws[chemical_columns].applymap(replace_concentration_values)
    dfws['risk_count'] = dfws[concentration_columns].apply(
        lambda row: sum(value in ['No apta como FAAP', 'Alta'] for value in row),axis=1
    )
    dfws['risk_concentration_sure'] = dfws.apply(lambda row: calculate_concentration_sum(dfws,row, iapc_mapping, ['1']), axis=1)
    dfws['risk_concentration_possible'] = dfws.apply(lambda row: calculate_concentration_sum(dfws,row, iapc_mapping, ['2B']), axis=1)
    df_water = dfws.groupby(['state_code','municipality_code']).agg({
        'risk_count': 'sum',
        'risk_concentration_sure': 'sum',
        'risk_concentration_possible': 'sum'
    }).reset_index()
    return df_water

In [27]:
st_activities = ["dumps", "biomass", "carpentry", "cement_gypsum", "sugarcane_cultivation", 
              "chemical_manufacturing", "pesticide_manufacturing", "gas_stations", 
              "incinerator", "sugar_mill", "brick_factory", "mining", "paper_mill", 
              "solvent_paint", "high_traffic_supply", "high_traffic_alameda", 
              "high_traffic_station"]
def stationary_sources():
    dfs = pd.read_csv("../dataset/Mexico/stationary_pollution_sources/stationary_pollution_sources.csv")
    dfs = process_df(dfs)
    gdf = geopd.read_file("../dataset/Mexico/geometry/2023_1_00_MUN/2023_1_00_MUN.shp")
    gdf_sp = geopd.GeoDataFrame(
        dfs, geometry=geopd.points_from_xy(dfs.x_coordinate,dfs.y_coordinate, crs="epsg:4326",
    ))
    combgdf = geopd.sjoin(gdf_sp, gdf, how="left", predicate="intersects")
    dfsp = combgdf
    dfsp['CVE_MUN'] = dfsp['CVE_MUN'].fillna('0')
    dfsp['CVE_MUN'] = dfsp['CVE_MUN'].astype(str).str.lstrip('0').replace('', '0').astype(int)
    dfsp['municipality_code'] = dfsp['CVE_MUN']
    dfsp['activity'] = dfsp['activity'].str.lower()
    dfsp2 = dfsp.pivot_table(index=['state_code', 'municipality_code'], 
                               columns='activity', 
                               aggfunc='size', 
                               fill_value=0)
    dfsp2 = dfsp2.reset_index()
    dfsp2 = dfsp.merge(dfsp2, on=['state_code', 'municipality_code'], how='left')
    dfsp2 = process_df(dfsp2)
    # dfsp2['st_risk'] = 1
    # df_sp_sources = dfsp2.groupby(['state_code', 'municipality_code'])[st_activities+['st_risk']].sum().reset_index()
    df_sp_sources = dfsp2.groupby(['state_code', 'municipality_code'])[st_activities].mean().reset_index()
    df_sp_sources['st_risk'] = df_sp_sources[st_activities].sum(axis=1)
    return df_sp_sources

In [ ]:
def fuels():
    df = pd.read_csv("../dataset/Mexico/fuels/fuels.csv")
    df = process_df(df)
    df_fuels = df.groupby(['state_code','municipality_code']).agg({
        'electricity_fuel': 'sum',
        'firewood_fuel': 'sum',
        'oil_fuel': 'sum',
    }).reset_index()
    return df_fuels

In [29]:
def health():
    df_hs = pd.read_csv("../dataset/Mexico/health_services/health_services.csv")
    df_hs = process_df(df_hs)
    df_hs['year'] = df_hs['registration_date'].apply(round_to_nearest_year)
    dfhs = df_hs.pivot_table(index=['state_code', 'municipality_code','year'], 
                               columns='service_type', 
                               aggfunc='size', 
                               fill_value=0)
    dfhs = dfhs.reset_index()
    dfhs = df_hs.merge(dfhs, on=['state_code', 'municipality_code','year'], how='left')
    dfhs = process_df(dfhs)
    dfhs2 = dfhs.groupby(['state_code','municipality_code','year']).agg({
        'support':'sum',
        'social_assistance':'sum',
        'outpatient_consultation':'sum',
        'hospitalization':'sum'
    }).reset_index()
    years = pd.DataFrame({'year': range(2008, 2023)})
    unique_codes = dfhs2[['state_code', 'municipality_code']].drop_duplicates()
    idx = unique_codes.assign(key=1).merge(years.assign(key=1), on='key').drop('key', axis=1)
    dfhs3 = idx.merge(dfhs2, on=['state_code', 'municipality_code', 'year'], how='left')
    dfhs3.fillna(0, inplace=True)
    dfhs2.sort_values(by=['state_code', 'municipality_code', 'year'], inplace=True)
    colscumsum = ['support', 'social_assistance', 'outpatient_consultation', 'hospitalization']
    for col in colscumsum:
        cumcol = f'{col}_cumsum'
        dfhs3[cumcol] = dfhs3.groupby(['state_code', 'municipality_code'])[col].cumsum()
    return dfhs3

,id,state_code,state_name,municipality_code,municipality_name,unit_name,institution_name,service_type,x_coordinate,y_coordinate,source_of_information,registration_date
38950,29287,32,ZACATECAS,57,TRANCOSO,TRANCOSO,INSTITUTO MEXICANO DEL SEGURO SOCIAL REGIMEN B...,DE CONSULTA EXTERNA,-102.364,22.7314,clues,2014-05-08
39311,40801,32,ZACATECAS,57,TRANCOSO,FUNDACION BEST A.C.,SERVICIOS MEDICOS PRIVADOS,DE CONSULTA EXTERNA,-102.3663,22.7341,clues,2017-10-03
39499,29739,32,ZACATECAS,57,TRANCOSO,HOSPITAL COMUNITARIO TRANCOSO,SECRETARIA DE SALUD,DE HOSPITALIZACIÓN,-102.36467588714731,22.73381633608664,clues,2011-05-10


,state_code,municipality_code,year,support,social_assistance,outpatient_consultation,hospitalization,support_cumsum,social_assistance_cumsum,outpatient_consultation_cumsum,hospitalization_cumsum
36780,32,57,2008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36781,32,57,2009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36782,32,57,2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36783,32,57,2011,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
36784,32,57,2012,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
36785,32,57,2013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
36786,32,57,2014,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
36787,32,57,2015,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
36788,32,57,2016,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
36789,32,57,2017,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [7]:
def marginalization():
    dfms = pd.read_csv("../dataset/Mexico/marginalization/IMU_2020.csv")
    dfms = process_df(dfms)
    dfms['state_code'] = dfms['ENT']
    dfms['municipality_code'] = dfms['MUN']
    dfms2 = dfms.groupby(['state_code','municipality_code']).agg({
        'IMN_2020': 'sum',
    }).reset_index()
    return dfms2

In [ ]:
def emissions_bc():
    dfem = pd.read_csv("../dataset/Mexico/emissions/emissions.csv")
    dfem = process_df(dfem)
    dfem['concentration'] = dfem.apply(lambda row: calculate_concentration_sum2(row, ['1','2A']), axis=1)
    df_smr = get_smr_data2(aggrcol)
    dfem2 = dfem.groupby(['state_code','municipality_code','year']).agg({
        'concentration': 'sum',
    }).reset_index()
    df_emissions = pd.merge(df_smr, dfem2, on=aggrcols, how='left')
    df_emissions.fillna({'concentration': 0}, inplace=True)
    return df_emissions

In [ ]:
def water_sources_bc():
    dfw = pd.read_csv("../dataset/Mexico/water_sources/water_sources.csv")
    dfw = process_df(dfw)
    gdf = geopd.read_file("../dataset/Mexico/geometry/2023_1_00_MUN/2023_1_00_MUN.shp")
    gdf_water = geopd.GeoDataFrame(
        dfw, geometry=geopd.points_from_xy(dfw.x_coordinate,dfw.y_coordinate, crs="epsg:4326",
    ))
    combgdf = geopd.sjoin(gdf_water, gdf, how="left", predicate="intersects")
    dfws = combgdf
    dfws['CVE_MUN'] = dfws['CVE_MUN'].astype(str).str.lstrip('0').replace('', '0').astype(int)
    dfws['municipality_code'] = dfws['CVE_MUN']
    # dfws['year'] = 2012
    concentration_columns = [col for col in dfws.columns if 'concentration' in col]
    chemical_columns = [col for col in dfws.columns if '_concentration' not in col]
    dfws[chemical_columns] = dfws[chemical_columns].applymap(replace_concentration_values)
    dfws['risk_count'] = dfws[concentration_columns].apply(
        lambda row: sum(value in ['No apta como FAAP', 'Alta'] for value in row),axis=1
    )
    dfws['risk_concentration_sure'] = dfws.apply(lambda row: calculate_concentration_sum(dfws,row, iapc_mapping, ['1']), axis=1)
    dfws['risk_concentration_possible'] = dfws.apply(lambda row: calculate_concentration_sum(dfws,row, iapc_mapping, ['2B']), axis=1)
    df_smr = get_smr_data2(aggrcols)
    dfws2 = dfws.groupby(['state_code','municipality_code']).agg({
        'risk_count': 'sum',
        'risk_concentration_sure': 'sum',
        'risk_concentration_possible': 'sum'
    }).reset_index()
    df_water = pd.merge(df_smr, dfws2, on=['state_code','municipality_code'], how='left')
    df_water.fillna({'risk_count': 0}, inplace=True)
    df_water.fillna({'risk_concentration_sure': 0}, inplace=True)
    df_water.fillna({'risk_concentration_possible': 0}, inplace=True)
    return df_water

In [ ]:
def stationary_sources_bc():
    dfs = pd.read_csv("../dataset/Mexico/stationary_pollution_sources/stationary_pollution_sources.csv")
    dfs = process_df(dfs)
    gdf = geopd.read_file("../dataset/Mexico/geometry/2023_1_00_MUN/2023_1_00_MUN.shp")
    gdf_sp = geopd.GeoDataFrame(
        dfs, geometry=geopd.points_from_xy(dfs.x_coordinate,dfs.y_coordinate, crs="epsg:4326",
    ))
    combgdf = geopd.sjoin(gdf_sp, gdf, how="left", predicate="intersects")
    dfsp = combgdf
    dfsp['CVE_MUN'] = dfsp['CVE_MUN'].fillna('0')
    dfsp['CVE_MUN'] = dfsp['CVE_MUN'].astype(str).str.lstrip('0').replace('', '0').astype(int)
    dfsp['municipality_code'] = dfsp['CVE_MUN']
    # dfsp['year'] = 2012
    dfsp['risk_count'] = 1
    df_smr = get_smr_data2('state_code')
    dfsp2 = dfsp.groupby(['state_code','municipality_code']).agg({
        'risk_count': 'sum',
    }).reset_index()
    df_sp_sources = pd.merge(df_smr, dfsp2, on=['state_code','municipality_code'], how='left')
    df_sp_sources.fillna({'risk_count': 0}, inplace=True)
    return df_sp_sources